# Link Prediction for Heterogeneous graph

## 0. Enviroment setup

In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm

from sklearn.metrics import roc_auc_score, f1_score
from torch_geometric.utils import negative_sampling
import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges
from torch_geometric.loader import LinkNeighborLoader

import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger

In [2]:
from link_prediction.models import *
from link_prediction.dataloaders import *

In [3]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [4]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## 1. Data Pre-processing

In [5]:
DATA_VERSION = 'final_v1'

In [6]:
data = getHiveDataset(
    f'dataset/hive/{DATA_VERSION}/nodes_labelled.csv',
    f'dataset/hive/{DATA_VERSION}/edges_labelled.csv',
)
data = T.ToUndirected()(data)

# data = torch.load('hive.pt')
data

HeteroData(
  user={
    x=[18645, 5],
    y=[18645],
  },
  comment={
    x=[125111, 1],
    y=[125111],
  },
  post={
    x=[13540, 1],
    y=[13540],
  },
  (user, upvote, comment)={
    edge_index=[2, 423638],
    edge_attr=[423638, 1],
    y=[423638],
  },
  (user, upvote, post)={
    edge_index=[2, 554131],
    edge_attr=[554131, 1],
    y=[554131],
  },
  (user, write, comment)={
    edge_index=[2, 78696],
    edge_attr=[78696, 1],
    y=[78696],
  },
  (user, write, post)={
    edge_index=[2, 12958],
    edge_attr=[12958, 1],
    y=[12958],
  },
  (user, downvote, comment)={
    edge_index=[2, 6819],
    edge_attr=[6819, 1],
    y=[6819],
  },
  (user, downvote, post)={
    edge_index=[2, 2934],
    edge_attr=[2934, 1],
    y=[2934],
  },
  (comment, belong_to, comment)={
    edge_index=[2, 117822],
    edge_attr=[117822, 1],
    y=[117822],
  },
  (comment, belong_to, post)={
    edge_index=[2, 19838],
    edge_attr=[19838, 1],
    y=[19838],
  },
  (comment, rev_upvote, user)

In [9]:
torch.save(data, 'hive.pt')

In [15]:
num_edges = len(data.edge_types)
edge_types = data.edge_types[:num_edges//2 + 1]

In [16]:
edge_types

[('user', 'upvote', 'comment'),
 ('user', 'upvote', 'post'),
 ('user', 'write', 'comment'),
 ('user', 'write', 'post'),
 ('user', 'downvote', 'comment'),
 ('user', 'downvote', 'post'),
 ('comment', 'belong_to', 'comment'),
 ('comment', 'belong_to', 'post')]

In [17]:
rev_edge_types = data.edge_types[num_edges//2 + 1:-1] + [edge_types[-2], data.edge_types[-1]] 

In [18]:
transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    disjoint_train_ratio=0.3, 
    add_negative_train_samples=False,
    # neg_sampling_ratio=2.0,
    edge_types=edge_types,
    rev_edge_types=rev_edge_types, 
)

train_data, val_data, test_data = transform(data)

## 2. Training

In [19]:
model = GATv2(
    in_channels=-1, 
    out_channels=128, 
    hidden_channels=[64, 128, 256, 256, 512], 
    metadata=data.metadata(), 
    # aggr_scheme='mean',
)


In [20]:
lr = 1e-3
optim = torch.optim.Adam(model.parameters(), lr=lr)
model.set_optimizer(optim)

wandb_logger = WandbLogger(
    project="LinkPrediction_finalv1",
    log_model='all',
    save_dir='results/log',
    name='gatv2_large',
    entity='ssc_project'
)

In [ ]:
# for edge_types, rev_edge_types in edges:
for edge_type in edge_types:
    loss_checkpoint_callback = ModelCheckpoint(
        monitor=f'{edge_type[1]}_val_loss',
        dirpath='results/checkpoints/graphsage/loss',
        filename=f'LinkPred-{{epoch:02d}}-{{{edge_type[1]}_val_loss:.2f}}',
        save_top_k=3,
        save_last=True,
        mode='min',
        every_n_epochs=1
    )
    roc_auc_checkpoint_callback = ModelCheckpoint(
        monitor=f'{edge_type[1]}_val_roc_auc',
        dirpath='results/checkpoints/graphsage/roc_auc',
        filename=f'LinkPred-{{epoch:02d}}-{{{edge_type[1]}_val_roc_auc:.2f}}',
        save_top_k=3,
        save_last=True,
        mode='max',
        every_n_epochs=1
    )
    trainer = L.Trainer(
        max_epochs=100,
        check_val_every_n_epoch=1,
        callbacks=[
            loss_checkpoint_callback, 
            roc_auc_checkpoint_callback,
        ],
        logger=wandb_logger
    )
    
    train_batch_size=128
    val_batch_size=val_data[edge_type]['edge_label'].size(0)
    model.set_trainval_info(edge_type, train_batch_size, val_batch_size)
    train_edge_label_index = train_data[edge_type].edge_label_index
    train_edge_label = train_data[edge_type].edge_label
    train_loader = LinkNeighborLoader(
        data=train_data,
        num_neighbors=[20, 20],
        neg_sampling_ratio=2.0,
        edge_label_index=(edge_type, train_edge_label_index),
        edge_label=train_edge_label,
        batch_size=train_batch_size,
        shuffle=True,
        pin_memory=True,
        num_workers=8,
    )

    val_edge_label_index = val_data[edge_type].edge_label_index
    val_edge_label = val_data[edge_type].edge_label
    val_loader = LinkNeighborLoader(
        data=val_data,
        num_neighbors=[20, 20],
        neg_sampling_ratio=None,
        edge_label_index=(edge_type, val_edge_label_index),
        edge_label=val_edge_label,
        batch_size=val_batch_size,
        shuffle=False,
        pin_memory=True,
        num_workers=8,
    )
    
    trainer.fit(model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
